# Boeing Data Science Challenge Problem

#### Suraj Greenlund
sugr7650@colorado.edu

404-200-3443

#### Problem Statement
Included in these materials please find two comma separated values (.csv) files: Training_dataset.csv
and Test_dataset.csv

Included in the training dataset is information on used cars previously sold. Each row corresponds to
one used car listing. The first column of the data contains a unique identifier for the listing. The next
twenty-six columns contain information on parameters relevant to the transaction, with those
parameters described in more detail in the appendix attached. Finally, the last two columns of the
“Training_dataset.csv” contain information on “Vehicle_Trim” and “Dealer_Listing_Price”, which
describe the trim of the vehicle involved in the sale, and the price at which the vehicle was listed by the
dealer.

Your challenge is to build one or more models, through whatever means you find most appropriate,
capable of predicting vehicle trim and dealer listing price given the other twenty-six variables provided.

#### Instructions
4. Once your model is built, use it to make predictions on EACH of the 1,000 vehicle listings
included in the “Test_dataset.csv” file. Your output should be a comma separated values (.csv)
file with one-thousand rows and three columns. The first column should be the unique identifier
for the listing. The second column should be your predicted value for vehicle trim. The third
column should be your predicted value for dealer listing price.

5. Please submit a brief write-up of no more than 500 words describing the approach you selected
and why. Please save your response as a PDF if possible. Please copy any source code from Step
2 and paste it as text into an appendix at the end of your write-up.

6. Return your submission to us by replying back to the original email before the date and time
specified in that email. Please attach the CSV containing your predictions from Step 4 and the
PDF containing your write-up and source code from step 5 to the email. It should contain ONLY two attachments: the CSV from step 4 and the PDF from Step 5. Please don’t resubmit any of
our original data back in your reply

## Initial Data Review
Notes on first glance:

1. The listing ID is not relevant for training, but we need to keep it for our output file
2. Everything should be converted to lower case, symbols need to be dropped. Some rows have a lot of blank entries
2. There is a large disparity between private and non-private sellers (majority of data is false for sellerispriv)
3. Ratings and reviews could make a difference, dealers that sell only higher-end cars might be more likely to have better reviews. However, this also is affected by the number of people who leave reviews, so we can use a beta distribution to normalize the values.
4. State might be irrelevant if we assume to automotive market is generally the same across the USA. Zip code might add too much noise (there are ~ 41k zipcodes in the USA total), but technically it could make a difference in terms of the general economic status of individual zip codes
5. Vehicle body can be dropped, since it is only SUV for both test and trains sets
6. There is a range of colors, it might be beneficial to use NLP techniques and set each word from the color as its own value (so that black might be a common color, but a fancier color name like Brilliant Black might cause the adjective to affect trim/price). Or, we can find the most common colors this way and coimpare the column to that
7. Lots of variety in how the drive trains and transmissions were entered(FWD vs Front Wheel Drive vs FRONT-WHEEL Drive)
8. Engine type acolumn might be tricky to work with
9. Features column is set up nicely for NLP techniques
10. use commas to delimit history column
11. All cadillac XT5 or jeepgrand cherokee, which means we can drop one of the two columns
12. Seller notes will be tricky to work with
13. Last two columns are trim and price. Unfortunately, some of these (236 for trim, 52 for price) are blanks. They should probably be dropped since we don't want to train the model to predict blanks (unless we are using two separate models, then only drop the relevant column for each class)


# Model 1

For our first model, we will go ahead and drop the seller notes, since that will be a bit harder to parse, and focus on building out a prototype and then introducing that column if time allows. 

### Data Prep
Data needs to be standardized and normalized
- lowercase everything, get rid of symbols, drop cols/rows as necessary


In [1]:
# imports
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import tqdm

In [2]:
# read the csv files
train_df=pd.read_csv('Training_DataSet.csv', header=0)
test_df=pd.read_csv('Test_Dataset.csv', header=0)

In [3]:
train_df = train_df.set_index('ListingID',verify_integrity=True)
test_df = test_df.set_index('ListingID',verify_integrity=True)

In [4]:
# drop columns 

# all are SUVs, so drop bodystyle, only 2 makes and 1 model for each, 
# so we can drop the make and know that xt5=cadillac and grand cherokee=jeep

# drop the state and city, assuming that the automotive market is 
# generally the same across the USA

# all vehicles are used, and all are 8-speed automative, so remove those column

# use zipcode instead of seller name, since they are, for the most part, 
# relatively unique to the seller
# so it's better formatted for an ml model

cols2drop=['VehBodystyle',
           'VehMake',
           'SellerCity',
           'SellerState',
           'SellerName',
           'SellerName',
           'VehType',
           'VehTransmission'
          ];

train_df = train_df.drop(columns=cols2drop)
test_df = test_df.drop(columns=cols2drop)

# drop the rows where our two classes have NaNs
train_df = train_df.dropna(subset=['Vehicle_Trim', 'Dealer_Listing_Price'])


In [5]:
# Since we need to consider both the rating and the number of reviews, we
# can use a beta distribution to normalize the data

prior_a = 3 #prior alpha
prior_b = 2 # prior beta

# replace the ratings column with the normalized ratings
train_df['SellerRating'] = (prior_a + train_df.SellerRevCnt * (train_df.SellerRating/5))/(prior_b+prior_a+train_df.SellerRevCnt)
test_df['SellerRating'] = (prior_a + test_df.SellerRevCnt * (test_df.SellerRating/5))/(prior_b+prior_a+test_df.SellerRevCnt)

# drop the review count column since we don't need it anymore
train_df = train_df.drop(columns=['SellerRevCnt'])
test_df = test_df.drop(columns=['SellerRevCnt'])

In [6]:
# columns to standardize 
# (convert to bool dtypes, remove symbols, lowercase, convert features to arrays, etc)
cols_std = ['VehColorExt',
        'VehColorInt',
        'VehDriveTrain',
        'VehEngine',
        'VehFeats',
        'VehFuel',
        'VehHistory',
];

bool_cols = ['SellerIsPriv',
         'VehCertified'
    
];


for dataset in [train_df,test_df]:
    # convert the columns with true/false into bool datatype
    for bools in bool_cols:
        dataset[bools] = dataset[bools].astype(bool)
    for col in cols_std:
        # lowercase strings, remove symbols 
        # replace semicolons with commas
        dataset[col] = dataset[col].str.lower()
        dataset[col] = dataset[col].str.strip('[]*)(&#')
        dataset[col] = dataset[col].str.replace(r"[\"\']", '')
        dataset[col] = dataset[col].str.replace(r";", ',')
        # replace slashes spaces for now
        dataset[col] = dataset[col].str.replace(r"/", ' ')


In [7]:
train_df.head()

,SellerIsPriv,SellerListSrc,SellerRating,SellerZip,VehCertified,VehColorExt,VehColorInt,VehDriveTrain,VehEngine,VehFeats,VehFuel,VehHistory,VehListdays,VehMileage,VehModel,VehPriceLabel,VehSellerNotes,VehYear,Vehicle_Trim,Dealer_Listing_Price
ListingID,,,,,,,,,,,,,,,,,,,,
3287,False,Inventory Command Center,0.945946,48091.0,False,white,black,4x4,3.6l v6,"adaptive cruise control, antilock brakes, audi...",gasoline,"1 owner, non-personal use reported, buyback pr...",8.600069,39319.0,Grand Cherokee,Fair Price,NaN,2015,High Altitude,30990.0
4777,False,Jeep Certified Program,0.958723,53186.0,True,brilliant black crystal pearlcoat,black,4x4 4wd,regular unleaded v-6 3.6 l 220,"18 wheel &amp, 8.4 radio group-inc: nav-capabl...",gasoline,"1 owner, buyback protection eligible",28.107014,38957.0,Grand Cherokee,Good Deal,Backed by a rigorous 125-point inspection by f...,2015,Laredo,23249.0
6242,False,Inventory Command Center,0.826154,63385.0,False,diamond black crystal pearlcoat,black,4wd,3.6l v6,"android auto, antilock brakes, apple carplay, ...",gasoline,"1 owner, non-personal use reported, buyback pr...",59.816875,20404.0,Grand Cherokee,Good Deal,Drop by to see us and you will quickly see how...,2018,Limited,31977.0
7108,False,HomeNet Automotive,0.731139,72703.0,False,radiant silver metallic,cirrus,fwd,gas v6 3.6l 222.6,"4-wheel disc brakes, abs, adjustable steering ...",gasoline,"1 owner, non-personal use reported, buyback pr...",98.665301,19788.0,XT5,Good Deal,"Luxury, Exterior Parking Camera Rear, Front Du...",2018,Luxury,33495.0
10882,False,Digital Motorworks (DMi),0.600000,14760.0,False,billet silver metallic clearcoat,black,4wd,3.6l v6 24v mpfi dohc,"1st and 2nd row curtain head airbags, 4-wheel ...",gasoline,"1 owner, non-personal use reported, buyback pr...",30.967500,34649.0,Grand Cherokee,Good Deal,Priced below KBB Fair Purchase Price! Clean CA...,2018,Limited,27900.0


In [8]:
# standardize drivetrain col
for dataset in [train_df, test_df]:
    # standardize 4wd
    # if a car is AWD and 4wd, we will call it 4wd (since that is toggleable whereas AWD is always on)
    dataset.loc[dataset['VehDriveTrain'].str.contains('4', case=False, na = False), 'VehDriveTrain'] = '4wd'
    dataset.loc[dataset['VehDriveTrain'].str.contains('four', case=False, na = False), 'VehDriveTrain'] = '4wd'
    
    # awd cases
    dataset.loc[dataset['VehDriveTrain'].str.contains('all', case=False, na = False), 'VehDriveTrain'] = 'awd'
    
    #fwd cases
    # assume if it is 2wd, that it is front-wheel and not rear wheel (from a quick google search)
    # for the single case in the training set that COULD occur in other datasets
    dataset.loc[dataset['VehDriveTrain'].str.contains('front', case=False, na = False), 'VehDriveTrain'] = 'fwd'
    dataset.loc[dataset['VehDriveTrain'].str.contains('2', case=False, na = False), 'VehDriveTrain'] = 'fwd'


In [9]:
# create a new class label for if the car is supercharged
# These could tend to be more expensive, since adding a supercharger is very expensive
# On the other hand, lots of people don't like to buy cars modded to that extent, driving the resale price back down
for dataset in [train_df, test_df]:
    dataset['Supercharged'] = dataset['VehEngine'].str.contains('super',case=False, na = False)

# set cols for number of cylinders
# ml models like bools :)
for dataset in [train_df, test_df]:
    dataset['V6'] = dataset['VehEngine'].str.contains('6',case=False, na = False)               
    dataset['V8'] = dataset['VehEngine'].str.contains('8',case=False, na = False)
    # some notes might have the model name in them... but we can take this part out if need be ;)
    dataset['ContainsTrailHawk'] = dataset['VehSellerNotes'].str.contains('trailhawk',case=False, na = False)  
    dataset['ContainsTrackhawk'] = dataset['VehSellerNotes'].str.contains('trackhawk',case=False, na = False)
    
# drop the seller notes because they will, for the most part, add noise
# later we can see the most common words for specific price ranges
train_df = train_df.drop(columns=['VehSellerNotes'])
test_df = test_df.drop(columns=['VehSellerNotes'])

#train_df = train_df.drop(columns=['VehEngine'])
#test_df = test_df.drop(columns=['VehEngine'])

In [10]:
# convert our trim class to numbers using a dictionary
# this makes for easy conversion back to a string when we output our predictions

trim_classes = train_df['Vehicle_Trim'].unique()
trim_dict={}
for i in range(len(trim_classes)):
    trim_dict[trim_classes[i]] = i
    
train_df=train_df.replace({"Vehicle_Trim": trim_dict})


,SellerIsPriv,SellerListSrc,SellerRating,SellerZip,VehCertified,VehColorExt,VehColorInt,VehDriveTrain,VehEngine,VehFeats,...,VehModel,VehPriceLabel,VehYear,Vehicle_Trim,Dealer_Listing_Price,Supercharged,V6,V8,ContainsTrailHawk,ContainsTrackhawk
ListingID,,,,,,,,,,,,,,,,,,,,,
3287,False,Inventory Command Center,0.945946,48091.0,False,white,black,4wd,3.6l v6,"adaptive cruise control, antilock brakes, audi...",...,Grand Cherokee,Fair Price,2015,0,30990.0,False,True,False,False,False
4777,False,Jeep Certified Program,0.958723,53186.0,True,brilliant black crystal pearlcoat,black,4wd,regular unleaded v-6 3.6 l 220,"18 wheel &amp, 8.4 radio group-inc: nav-capabl...",...,Grand Cherokee,Good Deal,2015,1,23249.0,False,True,False,False,False
6242,False,Inventory Command Center,0.826154,63385.0,False,diamond black crystal pearlcoat,black,4wd,3.6l v6,"android auto, antilock brakes, apple carplay, ...",...,Grand Cherokee,Good Deal,2018,2,31977.0,False,True,False,False,False
7108,False,HomeNet Automotive,0.731139,72703.0,False,radiant silver metallic,cirrus,fwd,gas v6 3.6l 222.6,"4-wheel disc brakes, abs, adjustable steering ...",...,XT5,Good Deal,2018,3,33495.0,False,True,False,False,False
10882,False,Digital Motorworks (DMi),0.600000,14760.0,False,billet silver metallic clearcoat,black,4wd,3.6l v6 24v mpfi dohc,"1st and 2nd row curtain head airbags, 4-wheel ...",...,Grand Cherokee,Good Deal,2018,2,27900.0,False,True,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8615125,True,Sell It Yourself,0.600000,48126.0,False,black,black,4wd,6-cylinder,NaN,...,Grand Cherokee,NaN,2015,0,18699.0,False,True,False,False,False
8615510,False,Digital Motorworks (DMi),0.645714,46280.0,False,gray,black,4wd,3.0l v6 24v ddi dohc turbo diesel,"1st and 2nd row curtain head airbags, 4-wheel ...",...,Grand Cherokee,Fair Price,2015,2,31998.0,False,True,False,False,False
8616294,False,Digital Motorworks (DMi),0.776000,43017.0,True,black,jet black,fwd,3.6l v6 24v gdi dohc,"1st and 2nd row curtain head airbags, 4-wheel ...",...,XT5,Good Deal,2018,3,35674.0,False,True,False,False,False


In [11]:
numeric_cols = ['VehYear',
               'VehMileage',
               'VehListdays',
               ];
cat_cols = ['SellerListSrc',
            'VehColorExt',
            'VehColorInt',
            'VehDriveTrain',
            'VehEngine',
            'VehFeats',
            'VehFuel',
            'VehHistory',
            'VehModel',
            'VehPriceLabel'
           ];

for dataset in [train_df, test_df]:
    for cols in cat_cols:
        dataset[cols].fillna('', inplace=True)

In [12]:
# normalize numeric inputs
scaler = MinMaxScaler()
scaler.fit(train_df[numeric_cols])

train_df[numeric_cols] = scaler.transform(train_df[numeric_cols])
test_df[numeric_cols] = scaler.transform(test_df[numeric_cols])


# one hot encoding categorical inputs

# the global encoder
encoder = OneHotEncoder(handle_unknown='ignore').fit(train_df[cat_cols])
new_one_hot_cols = [] #list(encoder.get_feature_names(features_cat_cols))

for column in cat_cols:
    encoder_i = OneHotEncoder(sparse=False, handle_unknown='ignore').fit(train_df[[column]])
    new_one_hot_cols_i = list(encoder_i.get_feature_names([column]))
    train_df[new_one_hot_cols_i] = encoder_i.transform(train_df[[column]])
    test_df[new_one_hot_cols_i] = encoder_i.transform(test_df[[column]])
    new_one_hot_cols += new_one_hot_cols_i



In [13]:
train_inputs = train_df[new_one_hot_cols + numeric_cols]
test_inputs = test_df[new_one_hot_cols + numeric_cols]

In [14]:
# define x and y
classes = ['Vehicle_Trim', 'Dealer_Listing_Price']


#predict y classes separately 
Y1 = train_df['Vehicle_Trim']
Y2 = train_df['Dealer_Listing_Price']

x_train, x_val, y_train1, y_val1 = train_test_split(train_inputs, Y1, train_size=0.75, random_state=78)
x_train, x_val, y_train2, y_val2 = train_test_split(train_inputs, Y2, train_size=0.75, random_state=78)




ListingID
3287        0
4777        1
6242        2
7108        3
10882       2
           ..
8615125     0
8615510     2
8616294     3
8617378     3
8620012    13
Name: Vehicle_Trim, Length: 5841, dtype: int64

In [15]:

encoder = OneHotEncoder(handle_unknown='ignore').fit(train_df[cat_cols])
new_one_hot_cols = [] #list(encoder.get_feature_names(features_cat_cols))

for column in cat_cols:
    encoder_i = OneHotEncoder(sparse=False, handle_unknown='ignore').fit(train_df[[column]])
    new_one_hot_cols_i = list(encoder_i.get_feature_names([column]))
    train_df[new_one_hot_cols_i] = encoder_i.transform(train_df[[column]])
    test_df[new_one_hot_cols_i] = encoder_i.transform(test_df[[column]])
    new_one_hot_cols += new_one_hot_cols

In [16]:
# prototype for a word filter
# find the most common words for all entries 
# then use those as stop words to find the most common words that are more unique to just the expensive cars

'''import nltk
from collections import Counter
import re

upper_df = train_df[train_df.Dealer_Listing_Price > 50000]

top_N = 200
stopwords = nltk.corpus.stopwords.words('english')
# RegEx for stopwords
RE_stopwords = r'\b(?:{})\b'.format('|'.join(stopwords))
# replace '|'-->' ' and drop all stopwords
words = (train_df.VehSellerNotes
           .str.lower()
           .replace([r'\|', RE_stopwords], [' ', ''], regex=True)
           .str.cat(sep=' ')
           .split()
)

# generate DF out of Counter
rslt = pd.DataFrame(Counter(words).most_common(top_N),
                    columns=['Word', 'Frequency']).set_index('Frequency')

newStopWords = rslt.values.flatten()


topN = 50
stopwords2 = nltk.corpus.stopwords.words('english')
stopwords2.extend(newStopWords)
# RegEx for stopwords
RE_stopwords2 = r'\b(?:{})\b'.format('|'.join(map(re.escape, stopwords2)))
# replace '|'-->' ' and drop all stopwords
words2 = (upper_df.VehSellerNotes
           .str.lower()
           .replace([r'\*', RE_stopwords2], [' ', ''], regex=True)
           .str.cat(sep=' ')
           .split()
)

# generate DF out of Counter
rslt2 = pd.DataFrame(Counter(words2).most_common(topN),
                    columns=['Word', 'Frequency']).set_index('Frequency')'''

"import nltk\nfrom collections import Counter\nimport re\n\nupper_df = train_df[train_df.Dealer_Listing_Price > 50000]\n\ntop_N = 200\nstopwords = nltk.corpus.stopwords.words('english')\n# RegEx for stopwords\nRE_stopwords = r'\x08(?:{})\x08'.format('|'.join(stopwords))\n# replace '|'-->' ' and drop all stopwords\nwords = (train_df.VehSellerNotes\n           .str.lower()\n           .replace([r'\\|', RE_stopwords], [' ', ''], regex=True)\n           .str.cat(sep=' ')\n           .split()\n)\n\n# generate DF out of Counter\nrslt = pd.DataFrame(Counter(words).most_common(top_N),\n                    columns=['Word', 'Frequency']).set_index('Frequency')\n\nnewStopWords = rslt.values.flatten()\n\n\ntopN = 50\nstopwords2 = nltk.corpus.stopwords.words('english')\nstopwords2.extend(newStopWords)\n# RegEx for stopwords\nRE_stopwords2 = r'\x08(?:{})\x08'.format('|'.join(map(re.escape, stopwords2)))\n# replace '|'-->' ' and drop all stopwords\nwords2 = (upper_df.VehSellerNotes\n           .str.l